In [42]:
from configs import MovieLens100K as ds
ds['ds_name']

'MovieLens100K'

In [35]:
# importing required pakcages
import pandas as pd
from tqdm import tqdm

In [43]:
dataset = pd.read_csv(f"datasets/{ds['ds_name']}/ratings_data.txt", sep=ds['ds_sep'], names=ds['ds_names'])

In [44]:
dataset.head()

,uid,iid,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [45]:
# what columns or attributes the dataset is included?
dataset.columns

Index(['uid', 'iid', 'rating', 'timestamp'], dtype='object')

In [46]:
# statistics on explicit dataset
print("Original dataset statistics: ")
print(f"> No. of users: {len(dataset['uid'].unique())}")
print(f"> No. of Items: {len(dataset['iid'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

Original dataset statistics: 
> No. of users: 943
> No. of Items: 1682
> No. of Interaction: 100000


In [47]:
# checking the number of times in which a rating happened - (0: implicit, 1-10: explicit)
dataset['rating'].value_counts()

4    34174
3    27145
5    21201
2    11370
1     6110
Name: rating, dtype: int64

In [48]:
# This method return a dataframe in which the specified values are removed from a specific column. We can use it to generate the implicit/explicit dataset.
# To do this, we can remove the 0 or 1-10 values from the `Book-Rating` column.
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [49]:
# To remove the users with fewer than 5 interaction we first count the number of interactino per user and add a new column (`Count`) in the dataframe.
# This column shows the number of interaction per user in the dataset
users_counts = dataset['uid'].value_counts()
users_counts = users_counts.to_dict() #converts to dictionary
dataset['count'] = dataset['uid'].map(users_counts)

### Iteratively remove users and items with fewer than `ds_rate` interactions

In [50]:
user_interaction, item_interaction = 1, 1

while user_interaction != 0 or item_interaction != 0:
    print(f"The current number of user and item with < {ds['ds_core']} interactions: ")
    # user side fewer than ds['ds_core'] cheking
    uid_value_counts = dataset['uid'].value_counts()
    user_interaction = uid_value_counts[uid_value_counts < ds['ds_core']].count()
    print(f"No. of users < {ds['ds_core']} ineractions: {user_interaction}")

    users_counts = dataset['uid'].value_counts()
    users_counts = users_counts.to_dict() #converts to dictionary
    dataset['count'] = dataset['uid'].map(users_counts)

    dataset = filter_rows_by_values(dataset, "count", list(range(ds['ds_core'])))

    # item side fewer than ds_rate cheking
    bid_value_counts = dataset['iid'].value_counts()
    item_interaction = bid_value_counts[bid_value_counts < ds['ds_core']].count()
    print(f"No. of items < {ds['ds_core']} ineractions: {item_interaction}")

    items_counts = dataset['iid'].value_counts()
    items_counts = items_counts.to_dict() #converts to dictionary
    dataset['count'] = dataset['iid'].map(items_counts)

    dataset = filter_rows_by_values(dataset, "count", list(range(ds['ds_core'])))

The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 0
No. of items < 5 ineractions: 333
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 0
No. of items < 5 ineractions: 0


In [51]:
# statistics on 5 rate explicit dataset (after pre-processing)
print(f"No. of users: {len(dataset['uid'].unique())}")
print(f"No. of Items: {len(dataset['iid'].unique())}")
print(f"No. of Interaction: {dataset.shape[0]}")

No. of users: 943
No. of Items: 1349
No. of Interaction: 99287


In [54]:
# Before we save the preprocessed explicit dataset (5Rate) we first remove the added column which is `Count`
del dataset['count']
if 'timestamp' in ds['ds_names']:
    del dataset['timestamp']

### User and Item ID Mapping

In [56]:
uid_to_index = dict()
iid_to_index = dict()

last_user_id = 0
last_item_id = 0

for eachline in tqdm(dataset.iterrows()):
  # add a new user id with an index
  if eachline[1][0] not in uid_to_index.keys():
    uid_to_index[eachline[1][0]] = last_user_id
    last_user_id += 1
  # add a new book id with an index
  if eachline[1][1] not in iid_to_index.keys():
    iid_to_index[eachline[1][1]] = last_item_id
    last_item_id += 1

99287it [00:10, 9062.75it/s]


In [57]:
# write the file with new mapped indices into a txt file
# mapped_dataset = open(f"datasets/{ds['ds_name']}/{ds['ds_name']}_data.csv", 'w')
mapped_dataset_txt = open(f"datasets/{ds['ds_name']}/{ds['ds_name']}_data.txt", 'w')
# mapped_dataset.write('uid' + ',' + 'bid' + ',' + 'rating' + '\n')

for eachline in tqdm(dataset.iterrows()):
    # mapped_dataset.write(str(uid_to_index[eachline[1][0]]) + "," + str(iid_to_index[eachline[1][1]]) + "," + str(eachline[1][2]) + "\n")
    mapped_dataset_txt.write(str(uid_to_index[eachline[1][0]]) + "," + str(iid_to_index[eachline[1][1]]) + "," + str(eachline[1][2]) + "\n")
# mapped_dataset.close()
mapped_dataset_txt.close()

99287it [00:12, 7692.25it/s]
